In [3]:
def KSA(key):
    key_length=len(key)
    S=list(range(256))
    j=0
    for i in range(256):
        j=(j+S[i]+key[i % key_length])%256
        S[i], S[j]=S[j], S[i]
    return S

def PRGA(S,n):
    i=0
    j=0
    key=[]
    while n>0:
        n=n-1
        i=(i+1)% 256
        j=(j+S[i])%256
        S[i], S[j]=S[j], S[i]
        K= S[(S[i]+S[j])%256]
        key.append(K)
    return key

key='divija'
plaintext="hello world"

def key_array(s):
    return [ord(c) for c in s]

key=key_array(key)

import numpy as np
S=KSA(key)

keystream=np.array(PRGA(S,len(plaintext)))
print(keystream)

plaintext=np.array([ord(i) for i in plaintext])
cipher=keystream ^ plaintext

print(cipher.astype(np.uint8).data.hex())
print([chr(c) for c in cipher])
        


[117 101 237  63  45 249 216 149 169 235 175]
1d00815342d9affadb87cb
['\x1d', '\x00', '\x81', 'S', 'B', 'Ù', '¯', 'ú', 'Û', '\x87', 'Ë']


In [1]:
import codecs

MOD = 256


def KSA(key):
    ''' Key Scheduling Algorithm (from wikipedia):
        for i from 0 to 255
            S[i] := i
        endfor
        j := 0
        for i from 0 to 255
            j := (j + S[i] + key[i mod keylength]) mod 256
            swap values of S[i] and S[j]
        endfor
    '''
    key_length = len(key)
    # create the array "S"
    S = list(range(MOD))  # [0,1,2, ... , 255]
    j = 0
    for i in range(MOD):
        j = (j + S[i] + key[i % key_length]) % MOD
        S[i], S[j] = S[j], S[i]  # swap values

    return S


def PRGA(S):
    ''' Psudo Random Generation Algorithm (from wikipedia):
        i := 0
        j := 0
        while GeneratingOutput:
            i := (i + 1) mod 256
            j := (j + S[i]) mod 256
            swap values of S[i] and S[j]
            K := S[(S[i] + S[j]) mod 256]
            output K
        endwhile
    '''
    i = 0
    j = 0
    while True:
        i = (i + 1) % MOD
        j = (j + S[i]) % MOD

        S[i], S[j] = S[j], S[i]  # swap values
        K = S[(S[i] + S[j]) % MOD]
        yield K


def get_keystream(key):
    ''' Takes the encryption key to get the keystream using PRGA
        return object is a generator
    '''
    S = KSA(key)
    return PRGA(S)


def encrypt_logic(key, text):
    ''' :key -> encryption key used for encrypting, as hex string
        :text -> array of unicode values/ byte string to encrpyt/decrypt
    '''
    # For plaintext key, use this
    key = [ord(c) for c in key]
    # If key is in hex:
    # key = codecs.decode(key, 'hex_codec')
    # key = [c for c in key]
    keystream = get_keystream(key)

    res = []
    for c in text:
        val = ("%02X" % (c ^ next(keystream)))  # XOR and taking hex
        res.append(val)
    return ''.join(res)


def encrypt(key, plaintext):
    ''' :key -> encryption key used for encrypting, as hex string
        :plaintext -> plaintext string to encrpyt
    '''
    plaintext = [ord(c) for c in plaintext]
    return encrypt_logic(key, plaintext)


def decrypt(key, ciphertext):
    ''' :key -> encryption key used for encrypting, as hex string
        :ciphertext -> hex encoded ciphered text using RC4
    '''
    ciphertext = codecs.decode(ciphertext, 'hex_codec')
    res = encrypt_logic(key, ciphertext)
    return codecs.decode(res, 'hex_codec').decode('utf-8')


def main():

    key = 'not-so-random-key'  # plaintext
    print("Enter your string to encrypt and then decrypt")
    plaintext = input("")  # plaintext
    # encrypt the plaintext, using key and RC4 algorithm
    ciphertext = encrypt(key, plaintext)
    print('plaintext:', plaintext)
    print('ciphertext:', ciphertext)
  
    
    decrypted = decrypt(key, ciphertext)
    print('decrypted:', decrypted)

    if plaintext == decrypted:
        print('\nEncryption and then DEcryption done')
    else:
        print('Something went wrong, please check the code and run again .-.')

    # until next time folks !



if __name__ == '__main__':
    main()

Enter your string to encrypt and then decrypt
divija
plaintext: divija
ciphertext: 0E79F774B5D8
decrypted: divija

Encryption and then DEcryption done
